# Particle Swarm Optimisation

## -> Sphere Function

In [11]:
import numpy as np
import itertools

The following will be the goal ("fitness") function. Here it is to be minimised.

In [16]:
def sphere_function(pos,dim):
    big_sum = 0
    for i in range(dim):
        big_sum = big_sum + pos[i]**2
    return big_sum

In [17]:
class Particle: # all the material that is relavant at the level of the individual particles
    
    def __init__(self, dim, minx, maxx):
        self.position = np.random.uniform(low=minx, high=maxx, size=dim)
        self.velocity = np.random.uniform(low=-0.1, high=0.1, size=dim)
        self.best_particle_pos = self.position
        self.dim = dim

        self.fitness = sphere_function(self.position,dim)
        self.best_particle_fitness = self.fitness   # we couldd start with very large number here, 
                                                    #but the actual value is better in case we are lucky 
                
    def setPos(self, pos):
        self.position = pos
        self.fitness = sphere_function(self.position,self.dim)
        if self.fitness<self.best_particle_fitness:     # to update the personal best both 
                                                        # position (for velocity update) and
                                                        # fitness (the new standard) are needed
                                                        # global best is update on swarm leven
            self.best_particle_fitness = self.fitness
            self.best_particle_pos = pos

    def updateVel(self, inertia, a1, a2, best_self_pos, best_swarm_pos):
                # Here we use the canonical version
                # V <- inertia*V + a1r1 (peronal_best - current_pos) + a2r2 (global_best - current_pos)
        cur_vel = self.velocity
        r1 = np.random.uniform(low=0, high=1, size = self.dim)
        r2 = np.random.uniform(low=0, high=1, size = self.dim)
        a1r1 = np.multiply(a1, r1)
        a2r2 = np.multiply(a2, r2)
        best_self_dif = np.subtract(best_self_pos, self.position)
        best_swarm_dif = np.subtract(best_swarm_pos, self.position)
                    # the next line is the main equation, namely the velocity update, 
                    # the velocities are added to the positions at swarm level 
        return inertia*cur_vel + np.multiply(a1r1, best_self_dif) + np.multiply(a2r2, best_swarm_dif)


In [18]:
class PSO: # all the material that is relavant at swarm leveel

    def __init__(self, w, a1, a2, dim, population_size, time_steps, search_range):

        # Here we use values that are (somewhat) known to be good
        # There are no "best" parameters (No Free Lunch), so try using different ones
        # There are several papers online which discuss various different tunings of a1 and a2
        # for different types of problems
        self.w = w # Inertia
        self.a1 = a2 # Attraction to personal best
        self.a2 = a2 # Attraction to global best
        self.dim = dim

        self.swarm = [Particle(dim,-search_range,search_range) for i in range(population_size)]
        self.time_steps = time_steps
        print('init')

        # Initialising global best, you can wait until the end of the first time step
        # but creating a random initial best and fitness which is very high will mean you
        # do not have to write an if statement for the one off case
        self.best_swarm_pos = np.random.uniform(low=-500, high=500, size=dim)
        self.best_swarm_fitness = 1e100

    def run(self):
        for t in range(self.time_steps):
            for p in range(len(self.swarm)):
                particle = self.swarm[p]

                new_position = particle.position + particle.updateVel(self.w, self.a1, self.a2, particle.best_particle_pos, self.best_swarm_pos)
                                
                if new_position@new_position > 1.0e+18: # The search will be terminated if the distance 
                                                        # of any particle from center is too large
                    print('Time:', t,'Best Pos:',self.best_swarm_pos,'Best Fit:',self.best_swarm_fitness)
                    raise SystemExit('Most likely divergent: Decrease parameter values')
 
                self.swarm[p].setPos(new_position)

                new_fitness = sphere_function(new_position,self.dim)

                if new_fitness < self.best_swarm_fitness:   # to update the global best both 
                                                            # position (for velocity update) and
                                                            # fitness (the new group norm) are needed
                    self.best_swarm_fitness = new_fitness
                    self.best_swarm_pos = new_position

            if t % 100 == 0: #we print only two components even it search space is high-dimensional
                print("Time: %6d,  Best Fitness: %14.6f,  Best Pos: %9.4f,%9.4f" % (t,self.best_swarm_fitness,self.best_swarm_pos[0],self.best_swarm_pos[1]), end =" ")
                if self.dim>2: 
                    print('...')
                else:
                    print('')

In [19]:
variable = PSO(dim=6, w=0.7, a1=2.02, a2=2.02, population_size=60, time_steps=1001, search_range=100).run()

init
Time:      0,  Best Fitness:    3337.989405,  Best Pos:  -12.2649,  -2.3836 ...
Time:    100,  Best Fitness:       0.000852,  Best Pos:   -0.0080,   0.0118 ...
Time:    200,  Best Fitness:       0.000555,  Best Pos:   -0.0116,  -0.0056 ...
Time:    300,  Best Fitness:       0.000555,  Best Pos:   -0.0116,  -0.0056 ...
Time:    400,  Best Fitness:       0.000555,  Best Pos:   -0.0116,  -0.0056 ...
Time:    500,  Best Fitness:       0.000348,  Best Pos:   -0.0037,   0.0010 ...
Time:    600,  Best Fitness:       0.000348,  Best Pos:   -0.0037,   0.0010 ...
Time:    700,  Best Fitness:       0.000348,  Best Pos:   -0.0037,   0.0010 ...
Time:    800,  Best Fitness:       0.000348,  Best Pos:   -0.0037,   0.0010 ...
Time:    900,  Best Fitness:       0.000348,  Best Pos:   -0.0037,   0.0010 ...
Time:   1000,  Best Fitness:       0.000340,  Best Pos:   -0.0159,  -0.0004 ...


In [22]:
import itertools
lst = list(itertools.product([0, 1], repeat=3))
lst

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [41]:
def change_singns(params,signs):
    changed_params = []
    for i in range(len(params)-1):
        if signs[i] == 1:
            changed_params.append(params[i])
        else:
            changed_params.append(-params[i])
    changed_params.append(params[2])
    return changed_params

# defining a list of tuples (w, alpha1, alpha2) 
params = (0.7,0.52,0.52)
lst_signs = list(itertools.product([0, 1], repeat=3))

params_lst = [change_singns(params,signs) for signs in lst_signs]
# params_lst = params_lst[1:]
params_lst

[[-0.7, -0.52, 0.52],
 [-0.7, -0.52, 0.52],
 [-0.7, 0.52, 0.52],
 [-0.7, 0.52, 0.52],
 [0.7, -0.52, 0.52],
 [0.7, -0.52, 0.52],
 [0.7, 0.52, 0.52],
 [0.7, 0.52, 0.52]]

In [42]:
def automator_signs(param_lst):
    for params in param_lst:
        print(params)
        weight = params[0]
        alpha1 = params[1]
        alpha2 = params[2]
        variable = PSO(dim=6, w=weight, a1=alpha1, a2=alpha2, population_size=21, time_steps=1001, search_range=100).run()
#     print(variable)
    
automator_signs(params_lst)

[-0.7, -0.52, 0.52]
init
Time:      0,  Best Fitness:    4376.264893,  Best Pos:    3.6946,  31.9217 ...
Time:    100,  Best Fitness:       2.011841,  Best Pos:   -0.0438,   0.6405 ...
Time:    200,  Best Fitness:       0.081121,  Best Pos:   -0.0576,  -0.0748 ...
Time:    300,  Best Fitness:       0.081121,  Best Pos:   -0.0576,  -0.0748 ...
Time:    400,  Best Fitness:       0.024855,  Best Pos:    0.0513,  -0.0448 ...
Time:    500,  Best Fitness:       0.024855,  Best Pos:    0.0513,  -0.0448 ...
Time:    600,  Best Fitness:       0.024855,  Best Pos:    0.0513,  -0.0448 ...
Time:    700,  Best Fitness:       0.024855,  Best Pos:    0.0513,  -0.0448 ...
Time:    800,  Best Fitness:       0.024855,  Best Pos:    0.0513,  -0.0448 ...
Time:    900,  Best Fitness:       0.024855,  Best Pos:    0.0513,  -0.0448 ...
Time:   1000,  Best Fitness:       0.024855,  Best Pos:    0.0513,  -0.0448 ...
[-0.7, -0.52, 0.52]
init
Time:      0,  Best Fitness:    2839.033737,  Best Pos:   12.6182,   1